In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [2]:
%matplotlib inline
import collections
import functools

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from joblib import Memory

from utils import import_data
from utils import wagering_stategy
from winning_horse_models.logistic_regression import LogisticRegressionModel
from constants import PMU_BETTINGS, SOURCE_PMU
from utils.wagering_stategy import compute_expected_return, compute_scenario, plot_scenario
import wagering_stategies
from wagering_stategies import factories as wagering_factories
from constants import CACHE_DIR

from matplotlib import style
style.use('ggplot')

memory = Memory(location=CACHE_DIR, verbose=0)

winning_model = LogisticRegressionModel.load_model(trainable=False)

In [ ]:
%%time



kelly_expected_return = compute_expected_return(
    compute_betting_fun=wagering_stategies.race_bettings_kelly,
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    winning_model=winning_model,
) 
# Save this dataframe
kelly_expected_return.to_csv('./data/kelly_expected_return.csv', index=False)

In [3]:
expected_return_df = pd.read_csv('./data/kelly_expected_return.csv')

In [4]:
# Plot expected return 
expected_return_df.mean()

betting_fraction                 0.048325
bet_size                         8.839215
expected_return                 -1.915370
relative_expected_return        -0.251542
computed_bets_on_n_horses        2.040406
actual_bets_on_n_horses          0.041244
n_horse                         11.590242
race_id                      95130.787828
dtype: float64

In [5]:
wagering_stategy.plot_expected_return(expected_return_df=expected_return_df)

On all races, your expected return is -25.15% (std: 4e+00)
You bet 4.08% of the time
Average not betting streaks: 2.5e+01 races
When you bet, on average your expected return is -25.15% (std: 4e+00)
When you bet, you win 3.49% of the time
When you bet, you lose 96.51% of the time
When you bet&win, you make 2044.11% of the time (std: 9e+00)
While betting, Average losing streaks 29.375 (std: 31.076267069904006)
While betting, Longest losing streak: 121
While betting, Longest winning streak: 2


In [3]:
@memory.cache
def compute_factory_expected_return(minimum_winning_proba:float, minimum_expected_return:float)->pd.DataFrame:
    return compute_expected_return(
        compute_betting_fun=wagering_factories.betting_on_best_exp_return_thresholded_winning_proba_expected_returns_factory(minimum_winning_probabilities=minimum_winning_proba, 
                                                                                                                                          expected_return_threshold=minimum_expected_return),
        source=SOURCE_PMU,
        code_pari="E_SIMPLE_GAGNANT",
        winning_model=winning_model,
    ) 

In [4]:
%%time
e_r_df = compute_factory_expected_return(minimum_winning_proba=0.3, minimum_expected_return=10)
wagering_stategy.plot_expected_return(expected_return_df=e_r_df)

On all races, your expected return is -100.00% (std: 0.0)
You bet 0.15% of the time(18 out of 11929 races)
Average not betting streaks: 626.89 races
When you bet, on average your expected return is -100.00% (std: 0.0)
When you bet, you win 0.00% of the time (0 out of 18 bets)
When you bet, you lose 100.00% of the time(18 out of 18 bets)
Not winning any bets!
While betting, Average losing streaks 18.0 (std: 0.0)
While betting, Longest losing streak: 18
There is no winning streaks!
CPU times: user 28.1 ms, sys: 8 ms, total: 36.1 ms
Wall time: 31.4 ms


In [5]:
e_r_df = compute_factory_expected_return(minimum_winning_proba=0.3, minimum_expected_return=6)
wagering_stategy.plot_expected_return(expected_return_df=e_r_df)

On all races, your expected return is +42.38% (std: 5.6)
You bet 0.43% of the time(51 out of 11929 races)
Average not betting streaks: 228.42 races
When you bet, on average your expected return is +42.38% (std: 5.6)
When you bet, you win 7.84% of the time (4 out of 51 bets)
When you bet, you lose 92.16% of the time(47 out of 51 bets)
When you bet&win, you make +1715.35% (std: 10.6)
While betting, Average losing streaks 9.4 (std: 7.1)
While betting, Longest losing streak: 20
While betting, Longest winning streak: 1


In [7]:
%%time
e_r_df = compute_factory_expected_return(minimum_winning_proba=0.1, minimum_expected_return=10)
wagering_stategy.plot_expected_return(expected_return_df=e_r_df)

On all races, your expected return is +34.92% (std: 10.3)
You bet 1.29% of the time(154 out of 11929 races)
Average not betting streaks: 77.47 races
When you bet, on average your expected return is +34.92% (std: 10.3)
When you bet, you win 1.95% of the time (3 out of 154 bets)
When you bet, you lose 98.05% of the time(151 out of 154 bets)
When you bet&win, you make +6825.87% (std: 33.3)
While betting, Average losing streaks 37.75 (std: 17.6)
While betting, Longest losing streak: 60
While betting, Longest winning streak: 1
CPU times: user 26.7 ms, sys: 4.95 ms, total: 31.7 ms
Wall time: 27.2 ms
